In [3]:
# general_loader.py
import json
import pandas as pd

In [9]:
from google.colab import drive

drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/SemEval-2026/train/eng_laptop_train_alltasks.jsonl'

data = []
with open(dataset_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

print(data[:2])  # preview first 2 entries


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[{'ID': 'laptop_quad_dev_1', 'Text': 'this unit is ` ` pretty ` ` and stylish , so my high school daughter was attracted to it for that reason .', 'Quadruplet': [{'Aspect': 'unit', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'pretty', 'VA': '7.12#7.12'}, {'Aspect': 'unit', 'Category': 'LAPTOP#DESIGN_FEATURES', 'Opinion': 'stylish', 'VA': '7.12#7.12'}]}, {'ID': 'laptop_quad_dev_2', 'Text': "for now i ' m okay with upping the experience & device to 3 out of 5 stars .", 'Quadruplet': [{'Aspect': 'device', 'Category': 'LAPTOP#GENERAL', 'Opinion': 'NULL', 'VA': '5.50#5.25'}]}]


In [4]:
def load_general_dataset(path):
    """
    Load SemEval dataset in general format (one row per quadruplet).

    Returns: DataFrame with columns:
    ID, Text, Aspect, Category, Opinion, VA, V, A
    """
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))

    records = []
    for row in data:
        for quad in row["Quadruplet"]:
            V, A = map(float, quad["VA"].split("#"))
            records.append({
                "ID": row["ID"],
                "Text": row["Text"],
                "Aspect": quad["Aspect"],
                "Category": quad["Category"],
                "Opinion": quad["Opinion"],
                "V": V,
                "A": A
            })

    return pd.DataFrame(records)

In [5]:
df = load_general_dataset("./train/eng_laptop_train_alltasks.jsonl")
df.shape

FileNotFoundError: [Errno 2] No such file or directory: './train/eng_laptop_train_alltasks.jsonl'

In [ ]:
# creating a dataset for task 1 with (text,aspect) as inputs and VA score as the label
df_task1 = df[["ID", "Text", "Aspect", "V", "A"]]
df_task1.to_csv("./train_csv/task1_dataset.csv", index=False)
print("Saved task1_dataset.csv (with V & A)")

Saved task1_dataset.csv (with V & A)


In [ ]:
# loading dataset from csv
df = pd.read_csv("./train_csv/task1_dataset.csv")

In [ ]:
df.Aspect.value_counts()

Aspect
laptop                  416
screen                  241
keyboard                219
chromebook              218
computer                216
                       ... 
build materials           1
bluetooth connection      1
travel                    1
google ecosystem          1
computer company          1
Name: count, Length: 955, dtype: int64

In [ ]:
df.head

<bound method NDFrame.head of                           ID  \
0          laptop_quad_dev_1   
1          laptop_quad_dev_1   
2          laptop_quad_dev_2   
3          laptop_quad_dev_3   
4          laptop_quad_dev_4   
...                      ...   
5768  laptop_quad_train_2932   
5769  laptop_quad_train_2933   
5770  laptop_quad_train_2933   
5771  laptop_quad_train_2933   
5772  laptop_quad_train_2934   

                                                   Text            Aspect  \
0     this unit is ` ` pretty ` ` and stylish , so m...              unit   
1     this unit is ` ` pretty ` ` and stylish , so m...              unit   
2     for now i ' m okay with upping the experience ...            device   
3     seems unlikely but whatever , i ' ll go with it .               NaN   
4     this version has been my least favorite versio...           version   
...                                                 ...               ...   
5768  an awesome product , well built - well w

In [ ]:
dataset.describe()